In [1]:
import PyLog as pl
import sys
sys.path.insert(0, '../')
import PyGol as pygol
from itertools import chain
from texttable import Texttable

In [2]:
BK_1 = ["starboard(X,Y) :- sector(X,Y,S), >=(S,1), =<(S,15)",
"port(X,Y)      :- sector(X,Y,S), >=(S,17), =<(S,31)",
"forward(X,Y)   :- sector(X,Y,S), >=(S,0), =<(S,7)",
"forward(X,Y)   :- sector(X,Y,S), >=(S,25), =<(S,31)",
"aft(X,Y)       :- sector(X,Y,S), >=(S,9), =<(S,23)", 
"ahead(X,Y) :- sector(X,Y,S), ==(S,0)",
    "bow(X,Y) :- sector(X,Y,S), >=(S,1), =<(S,4)",
    "broad(X,Y) :- sector(X,Y,S), ==(S,4)",
    "forward_beam(X,Y) :- sector(X,Y,S), >=(S,5), =<(S,7)",
    "beam(X,Y) :- sector(X,Y,S), ==(S,8)",
    "aft_beam(X,Y) :- sector(X,Y,S), >=(S,9), =<(S,11)",
    "broad(X,Y) :- sector(X,Y,S), ==(S,12)",
    "quarter(X,Y) :- sector(X,Y,S), >=(S,12), =<(S,15)",
    "astern(X,Y) :- sector(X,Y,S), ==(S,16)",
    "quarter(X,Y) :- sector(X,Y,S), >=(S,17), =<(S,20)",
    "broad(X,Y) :- sector(X,Y,S), ==(S,20)",
    "aft_beam(X,Y) :- sector(X,Y,S), >=(S,21), =<(S,23)",
    "beam(X,Y) :- sector(X,Y,S), ==(S,24)",
    "forward_beam(X,Y) :- sector(X,Y,S), >=(S,25), =<(S,27)",
    "broad(X,Y) :- sector(X,Y,S), ==(S,28)",
    "bow(X,Y) :- sector(X,Y,S), >=(S,28), =<(S,31)",

    "applies(X,Y,rule13_overtaking) :- risk_collision(Y,X), sector(Y,X,S), >=(S,10), =<(S,22)",
    "applies(X,Y,rule14_headon) :- risk_collision(X,Y), sector(X,Y,S1), ==(S1,0), sector(Y,X,S2), ==(S2,0)",
    "applies(X,Y,rule15_crossing) :- risk_collision(X,Y), sector(X,Y,S), >=(S,2), =<(S,9)",
    "applies(X,Y,rule15_crossing) :- risk_collision(X,Y), sector(X,Y,S), >=(S,23), <(S,30)",
    #"applies(X,Y,rule16_giveway) :- applies(X,Y,rule13_overtaking), aft(X,Y)",
    #"applies(X,Y,rule16_giveway) :- applies(X,Y,rule15_crossing), starboard(X,Y)",
    #"applies(X,Y,rule16_giveway) :- applies(X,Y,rule14_crossing)",
    #"applies(X,Y,rule17_standon) :- applies(X,Y,rule13_overtaking), aft(Y,X)",
    #"applies(X,Y,rule17_standon) :- applies(X,Y,rule15_crossing), port(X,Y)",
    "applies(X,Y, rule9_narrow) :- sector(X, Y, S), constraint_narrow(Y)",
    "applies(X,Y, rule18_privilege) :-  sector(X, Y, S), constraint_fishing(Y)",
    "applies(X,Y, rule18_privilege) :-  sector(X, Y, S), constraint_draught(Y)"
 
]

In [3]:
scenario = ["sector(v1, v2, 15)", "risk_collision(v1, v2)", "sector(v1, v3, 5)", "risk_collision(v1, v3)",
            "sector(v4,v5, 10)", "risk_collision(v4,v5)", "sector(v4, v6, 29)","risk_collision(v4, v6)",
            "sector(v7,v8, 15)", "risk_collision(v7,v8)", "sector(v7, v9,7)"
            ]

pos_example = ["priority(v1,v3,rule16_giveway,rule17_standon,rule16_giveway)",
               "priority(v4,v6,rule16_giveway,rule17_standon,rule16_giveway)",
             ]

neg_example = [
               "priority(v7,v8,rule16_giveway,rule17_standon,rule16_giveway)",
               ]

In [4]:
kb_1 = pl.KnowledgeBase()
kb_1.add_clauses(BK_1)
kb_1.add_clauses(scenario)



P1, N1 = pygol.relevant_literals(file=scenario, constant_set = [],  container = "memory",
                                positive_example=pos_example,
                                negative_example= neg_example, tqdm_disable=True) 

P2 = pl.modify_bcrl(P1, BK_1, [])
N2 = pl.modify_bcrl(N1, BK_1, [])




finak_bk_1 = list(chain.from_iterable(P2.values()))
finak_bk_2 = list(chain.from_iterable(N2.values()))

const = ["rule16_giveway", "rule17_standon","rule16_giveway", "rule15_crossing","rule18_privilege", "rule13_overtaking" ]
constr = [ "sector", "risk_collision", "constraint_draught",]
finak_bk = finak_bk_1+finak_bk_2

P, N = pygol.bottom_clause_generation(file=finak_bk,  
                                       constant_set = const ,  
                                       container = "memory",
                                positive_example=pos_example, 
                                negative_example=neg_example, constrains=constr,tqdm_disable=True)



Train_P = {i:j for i,j in P.items()}
Train_N = {i:j for i,j in N.items()}

model_1 = pygol.pygol_learn(Train_P, Train_N,  constant_set = const,  max_literals=3,  exact_literals=True, 
                    key_size=len(Train_P), min_pos=1, distinct=False, eval_metrics=False, verbose=True)
print("Learned Model: ")
print(model_1.hypothesis)
print(model_1.time_learn)

Positive Example0
	 priority(A,B,rule16_giveway,rule17_standon,rule16_giveway):-forward_beam(A,B)&applies(C,A,rule13_overtaking)&applies(A,B,rule15_crossing)
	 priority(A,B,rule16_giveway,rule17_standon,rule16_giveway):-forward_beam(A,B)&starboard(A,B)&applies(C,A,rule13_overtaking)
	 priority(A,B,rule16_giveway,rule17_standon,rule16_giveway):-forward_beam(A,B)&starboard(A,B)&applies(A,B,rule15_crossing)
	 priority(A,B,rule16_giveway,rule17_standon,rule16_giveway):-forward(A,B)&forward_beam(A,B)&starboard(A,B)
	 priority(A,B,rule16_giveway,rule17_standon,rule16_giveway):-forward_beam(A,B)&quarter(A,C)&starboard(A,B)
	 priority(A,B,rule16_giveway,rule17_standon,rule16_giveway):-forward(A,B)&forward_beam(A,B)&applies(C,A,rule13_overtaking)
	 priority(A,B,rule16_giveway,rule17_standon,rule16_giveway):-forward(A,B)&forward_beam(A,B)&applies(A,B,rule15_crossing)
	 priority(A,B,rule16_giveway,rule17_standon,rule16_giveway):-forward_beam(A,B)&quarter(A,C)&applies(C,A,rule13_overtaking)
	 prio